In [ ]:
#　dev限定
import sys
sys.argv = [sys.argv[0]] 
sys.argv += ['--env', 'dev'] 


In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructType, StructField, BooleanType
import pyspark
from  pyspark.sql.functions import input_file_name
import pyspark.pandas as ps
from awsglue.utils import getResolvedOptions

# pandas udf サンプル
schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("age", StringType(), True),
    StructField("file_name", StringType(), True),
    StructField("age_sai", StringType(), True)
])

def subtract_mean(pdf):
    # pdf is a pandas.DataFrame
    ages = pdf.age
    return pdf.assign(age_sai=ages+"歳")

In [ ]:
# conf設定
spark.conf.set("spark.sql.execution.arrow.pyspark.fallback.enabled","true")

args = getResolvedOptions(sys.argv, ['env'])

# 環境分離 例
if args['env'] == 'dev':
    prefix='file://'
else:
    prefis='s3://'

# ファイルの読み込み例
spark_df = spark.read.csv(prefix + '/home/glue_user/workspace/jupyter_workspace/test.csv', header=True)
spark_df=spark_df.withColumn('file_name', input_file_name())

#guroup（id）ごとに何かの処理を適用
#pandas udfを呼んでいる
#pandas udf例
kekka_spark_df = spark_df.groupby("id").applyInPandas(subtract_mean, schema=schema)


# parquet書き込み（with snappy）
kekka_spark_df.write.option("compression", "snappy").mode('overwrite').parquet(prefix + '/home/glue_user/workspace/jupyter_workspace/data/2023-11-11/output/')

In [ ]:
spark.read.parquet(prefix + '/home/glue_user/workspace/jupyter_workspace/data/2023-11-11/output/').show()

In [ ]:
# sparkのdataframeで読み込む
spark_df=spark.read.csv(prefix + '/home/glue_user/workspace/jupyter_workspace/test2.csv.bz2', header=True)
spark_df.show()

In [ ]:

# 3種類のデータフレームが登場する
## pandas dataframe(pandas_df)
## spark dataframe(spark_df)
## pandas dataframe on spark(pandas_df_on_spark)

import numpy as np
import pandas as pd

# pandasのデータフレーム(普通のpandasnなので悪手)
#pandas_df = spark_df.select("*").toPandas()
# これはダメ（普通のpandas操作になってる）
#pandas_on_spark_df.asfreq()

# pandas dataframe on spark
# pandasはpandasでもこれはOK
import pyspark.pandas as ps
pandas_df_on_spark = ps.DataFrame({'id': range(10)}, index=range(10))
# pandasだけどsparknizeされる
pandas_df_on_spark = pandas_df_on_spark.to_spark(index_col='index')

#これはOK
#ちなみにfilterはspark native
pandas_df_on_spark = pandas_df_on_spark.filter("id > 5")
pandas_df_on_spark
pandas_df_on_spark.show()

In [ ]:
import math
import numpy as np
import pandas as pd
from typing import List
import logging
# geopyは存在しない
# from geopy.distance import geodesic

In [ ]:
pandas_df_on_spark = ps.DataFrame({'a': [1,2,3], 'b':[4,5,6]})
def pandas_plus(pdf):
    return pdf + 1

# transfrom型なのでSeries単位がinputになってoutputもSeries
pandas_df_on_spark.pandas_on_spark.transform_batch(pandas_plus)

In [ ]:
import pandas as pd
print(pd.__version__)

In [ ]:
def length(pdf) -> ps.DataFrame[int, [int]]:
    return pd.DataFrame([len(pdf)])

pandas_df_on_spark = ps.DataFrame({'A': range(1000)})

# apply型なのでDataframe単位がinputになってoutputもDataframe
pandas_df_on_spark.pandas_on_spark.apply_batch(length)  

In [ ]:
# pandas -> spark
pdf = spark_df.toPandas() 
pdf

In [ ]:
# spark_dfをpandas_on_sparkkへ変換する
psdf = spark_df.to_pandas_on_spark()

In [ ]:
#普通にpandasぽく使える（サポートされているなら）
# サポートされているものはpandas_udfに優先して使う
psdf.iloc[1]


In [ ]:
#次に置き換えを考える
#rolling -> windowで対応できる
from pyspark.sql.functions import *
from pyspark.sql.window import Window
avg_val = max(col("age")).over(Window.partitionBy(col("id")))
spark_df.withColumn("max_age",avg_val).show(truncate=False)

In [ ]:
# サポートしていないものはpandas udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, TimestampType

# Sparkセッションの作成
spark = SparkSession.builder.appName("example").getOrCreate()

# サンプルデータの作成
data = [(1, '2023-01-01', 10.0),
        (2, '2023-01-02', 20.0),
        (3, '2023-01-03', 30.0)]
schema = StructType([StructField("id", IntegerType(), True),
                     StructField("date", StringType(), True),
                     StructField("value", DoubleType(), True)])
schema2 = StructType([StructField("id", IntegerType(), True),
                     StructField("date", TimestampType(), True),
                     StructField("value", DoubleType(), True)])
df = spark.createDataFrame(data, schema=schema)

# 日付列をTimestamp型に変換
df = df.withColumn("date", df["date"].cast("timestamp"))

# Pandas UDFでasfreqメソッドを実行
def asfreq_pandas_udf(pdf):
    pdf['date'] = pd.to_datetime(pdf['date'])
    pdf = pdf.set_index('date').asfreq('W')  # ここでasfreqメソッドを呼び出す
    return pdf.reset_index()

result_df = df.groupby("id").applyInPandas(asfreq_pandas_udf, schema=schema2)

# 結果を表示
result_df.show()